idea what the data could look like
(17890902, {'nom': 'Conseils Generaux', 'ville': 'versaille', 'sequenceHistorique': ''})
(timestamp1, {'a':False, 'b':False, 'c':False}),
(timestamp2, {'a':False, 'b':True, 'c':False}),
(timestamp3, {'a':False, 'b':False, 'c':False}), 
(timestamp4, {'a':False, 'b':True, 'c':False}),
https://stackoverflow.com/questions/38465620/looking-for-an-efficient-way-to-store-history-data

### 1) RECUPERATION DU CONTENU DE LA PAGE WIKIPEDIA <br>
https://fr.wikipedia.org/wiki/Chronologie_de_la_R%C3%A9volution_fran%C3%A7aise <br>

In [1]:
# Importer les modules nécessaires
import requests

# Récupérer l'Url
r = requests.get("https://fr.wikipedia.org/wiki/Chronologie_de_la_R%C3%A9volution_fran%C3%A7aise")

# Enregistrer le contenu dans un fichier
with open("response.html", "w") as f:
    f.write(r.text)
    f.close() 


In [2]:
# Importer les modules nécessaires
from bs4 import BeautifulSoup
import json

# Lire le fichier contenant le contenu
with open("response.html", "r") as f:
 
# Parser l'html pour récupérer les <h2>, <h3> et <li> visés
    soup = BeautifulSoup(f, 'html.parser')

periode = []
annee = []
dateEtEvenement = []
dates = soup.find(class_="mw-body-content").find_all("li", class_=False)
for date in dates:
    if date.find_previous("h3") == None:
        periode.append(date.find_previous("h2").text)
        annee.append(0)
        dateEtEvenement.append(date.text)
    else:
        periode.append(date.find_previous("h2").text)
        annee.append(date.find_previous("h3").text)
        dateEtEvenement.append(date.text)

# Sauvegarder les fichiers
with open("periode.json", "w") as f:
    json.dump(periode, f)
    f.close() 
with open("annee.json", "w") as f:
    json.dump(annee, f)
    f.close() 
with open("dateEtEvenement.json", "w") as f:
    json.dump(dateEtEvenement, f)
    f.close() 

### 2) NETTOYAGE DES DONNEES <br>

In [14]:
# Importer les modules nécessaires
import pandas as pd
import json

# Lire les jsons et récupérer les listes
with open("periode.json", "r") as f:
    periode = json.load(f)
with open("annee.json", "r") as f:
    annee = json.load(f)
with open("dateEtEvenement.json", "r") as f:
    dateEtEvenement = json.load(f)
    
# Construire le dataframe 
df = pd.DataFrame ({'Période': periode, 'Année': annee, 'dateEtEvenement': dateEtEvenement})

df[["date","Evenement"]] = df["dateEtEvenement"].str.split(":", n = 1, expand = True)
print(df.head())

                Période Année  \
0  Prélude (année 1788)     0   
1  Prélude (année 1788)     0   
2  Prélude (année 1788)     0   
3  Prélude (année 1788)     0   
4  Prélude (année 1788)     0   

                                     dateEtEvenement         date  \
0  7 juin : « Journée des Tuiles » à Grenoble. Le...      7 juin    
1  14 juin : Les notables grenoblois se réunissen...     14 juin    
2  21 juillet : Réunion des états généraux du Dau...  21 juillet    
3  8 août : décision de convoquer les États génér...      8 août    
4  25 août : Brienne démissionne. Il est remplacé...     25 août    

                                           Evenement  
0   « Journée des Tuiles » à Grenoble. Les Grenob...  
1   Les notables grenoblois se réunissent à l'Hôt...  
2   Réunion des états généraux du Dauphiné à Vizi...  
3   décision de convoquer les États généraux pris...  
4   Brienne démissionne. Il est remplacé par Jacq...  


' WORKS\ndf2 = df.head(20)\ndf2[["date","Evenement"]] = df2["dateEtEvenement"].str.split(":", n = 1, expand = True)\n'

In [ ]:
"""
# Récupérer les index à supprimer
toDrop = df.loc[(df['Période'] == 'Notes et références') | (df['Période'] == 'Voir aussi')].index
df2 = df.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df2.describe())
#print(df2.info())

df2 = df2.drop([df2.index[193]                                    
,df2.index[194]                     
,df2.index[319]                            
,df2.index[320]                               
,df2.index[454]  
,df2.index[489]           
,df2.index[502]                       
,df2.index[504]])

#print(df2)
"""
"""
toDrop = df2.loc[(df2['dateEtEvenement'].str.contains('Liste'))].index
df2 = df2.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df3.describe())
#print(df3.info())
"""
"""
toDrop = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
#print(toDrop)
#df3 = df2.drop(toDrop)
#print(df2.head(10))
#print(df2.tail(10))
#print(df2.dtypes)
#print(df3.describe())
#print(df3.info())

dfGood = df3.loc[(df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
#print(dfGood)

dfBad = df3.loc[(~df3['dateEtEvenement'].str.contains(r'(.*?):'))] #.index
print(dfBad)


12 septembre 1793 : 
bataille d'Avesnes-le-Sec (en)
combat d'Haspres 


19 septembre : 
319                          bataille du Pont-Barré[7]  
320                              bataille de Torfou[7] 


193                                   combat d'Haspres  
194                     bataille d'Avesnes-le-Sec (en)  
319                          bataille du Pont-Barré[7]  
320                              bataille de Torfou[7]  
454  En février on conseille à Bonaparte de porter ...  
489           6 - 7 novembre bataille de Calliano (en)  
502                        Expédition d'Irlande (1796)  
504                        Expédition d'Irlande (1798)  
"""

"""
for i in df2['dateEtEvenement']: 
    if i.count(":") == 2:
        print(i)
    if i.count(":") == 0:
        print(i)

    
#find a way to split
#print(df2['Date et Evènement'].str.split(' : ', n=1, expand=True))
#df3.columns = ['column{}'.format(x+1) for x in df3.columns]
#print(df2)
#print(df2['Date et Evènement'].str.split(' : ', 1).tolist(), columns = ['Date', 'Evènement'])

"""
"""
toDrop =[]
for i in df2['dateEtEvenement']: 
    if i.count(":") == 2 | i.count(":") != 1 :
        ind = df2.loc[(df2[i])].index
        toDrop.append(ind)
"""
"""
for i in df2['dateEtEvenement']: 
    if i.count(":") != 1:
        print(i)
"""  
"""
print(toDrop)
#df2 = df2.drop(toDrop)
 """       
df2[['First','Last']] = df2.dateEtEvenement.str.split(r"(.*?):", n=1) #REGEX IMPORTANT DETERMINER PREMIER :

print(df2)
#bug

Source : <br>
https://www.geeksforgeeks.org/web-scraping-from-wikipedia-using-python-a-complete-guide/ <br>

In [ ]:
2) Save data in file <br>
https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file

In [ ]:
3) Plot as a a timeline <br>
https://matplotlib.org/stable/gallery/lines_bars_and_markers/timeline.html
https://coderzcolumn.com/tutorials/data-science/timeline-using-matplotlib